In [69]:
def whitening(sigma):
    ''' Calculate a whitening matrix for covariance matrix sigma. '''
    U, l, _ = linalg.svd(sigma)
    return U.dot( np.diag(l ** -0.5) )

In [70]:
from numpy import linalg

def cov(trials):
    ''' Calculate the covariance for each trial and return their average '''
    ntrials = trials.shape[2]
    covs = [ trials[:,:,i].dot(trials[:,:,i].T) / nsamples for i in range(ntrials) ]
    return np.mean(covs, axis=0)

In [71]:
def csp(trials_r, trials_f):
    '''
    Calculate the CSP transformation matrix W.
    arguments:
        trials_r - Array (channels x samples x trials) containing right hand movement trials
        trials_f - Array (channels x samples x trials) containing foot movement trials
    returns:
        Mixing matrix W
    '''
    cov_r = cov(trials_r)
    cov_f = cov(trials_f)
    P = whitening(cov_r + cov_f)
    B, _, _ = linalg.svd( P.T.dot(cov_f).dot(P) )
    W = P.dot(B)
    return W

In [72]:
import scipy.signal 
from scipy.signal import butter,lfilter

def bandpass(trials, lo, hi, sample_rate):
    '''
    Designs and applies a bandpass filter to the signal.
    
    Parameters
    ----------
    trials : 3d-array (channels x samples x trials)
        The EEGsignal
    lo : float
        Lower frequency bound (in Hz)
    hi : float
        Upper frequency bound (in Hz)
    sample_rate : float
        Sample rate of the signal (in Hz)
    
    Returns
    -------
    trials_filt : 3d-array (channels x samples x trials)
        The bandpassed signal
    '''

    # The iirfilter() function takes the filter order: higher numbers mean a sharper frequency cutoff,
    # but the resulting signal might be shifted in time, lower numbers mean a soft frequency cutoff,
    # but the resulting signal less distorted in time. It also takes the lower and upper frequency bounds
    # to pass, divided by the niquist frequency, which is the sample rate divided by 2:
    a, b = scipy.signal.iirfilter(4, [lo/(sample_rate/2.0), hi/(sample_rate/2.0)],ftype='butter')
    # Applying the filter to each trial
    ntrials = trials.shape[2]
    trials_filt = np.zeros((nchannels, nsamples, ntrials))
    for i in range(ntrials):
        trials_filt[:,:,i] = scipy.signal.filtfilt(a, b, trials[:,:,i], axis=1)
    
    return trials_filt

In [73]:
def logvar(trials):
    '''
    Calculate the log-var of each channel.
    
    Parameters
    ----------
    trials : 3d-array (channels x samples x trials)
        The EEG signal.
        
    Returns
    -------
    logvar - 2d-array (channels x trials)
        For each channel the logvar of the signal
    '''
    return np.log(np.var(trials, axis=1))

In [74]:
def apply_mix(W, trials):
    ''' Apply a mixing matrix to each trial (basically multiply W with the EEG signal matrix)'''
    ntrials = trials.shape[2]
    trials_csp = np.zeros((nchannels, nsamples, ntrials))
    for i in range(ntrials):
        trials_csp[:,:,i] = W.T.dot(trials[:,:,i])
    return trials_csp

In [75]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [76]:
import os
import numpy as np
import pandas as pd
import warnings
import scipy.io
warnings.filterwarnings('ignore')
d=r'/content/drive/MyDrive/Arm-Reaching/Session 1/EEG_session1_sub11_reaching_realMove.mat'

In [77]:
m = scipy.io.loadmat(d, struct_as_record=True)

In [78]:
ch=['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'ch9', 'ch10', 'ch11', 'ch12', 'ch13', 'ch14', 'ch15', 'ch16', 'ch17', 'ch18', 'ch19', 'ch20', 'ch21', 'ch22', 'ch23', 'ch24', 'ch25', 'ch26', 'ch27', 'ch28', 'ch29', 'ch30', 'ch31', 'ch32', 'ch33', 'ch34', 'ch35', 'ch36', 'ch37', 'ch38', 'ch39', 'ch40', 'ch41', 'ch42', 'ch43', 'ch44', 'ch45', 'ch46', 'ch47', 'ch48', 'ch49', 'ch50', 'ch51', 'ch52', 'ch53', 'ch54', 'ch55', 'ch56', 'ch57', 'ch58', 'ch59', 'ch60']

In [79]:
arr=[m[i] for i in ch ]  
arr=np.hstack(arr)
EEG=arr.T
nchannels, nsamples =EEG.shape

In [80]:
sample_rate = m['nfo']['fs'][0][0][0][0]
channel_names = [s[0] for s in m['nfo']['clab'][0][0][0]]
event_onsets = m['mrk'][0][0][0]
event_codes = m['mrk'][0][0][1]
labels = np.zeros((1, nsamples), int)
labels[0, event_onsets] = event_codes
cl_lab = [s[0] for s in m['nfo']['className'][0][0][0]]
nclasses = len(cl_lab)
nevents = len(event_onsets)

In [81]:
# Print some information
print('Shape of EEG:', EEG.shape)
print('Sample rate:', sample_rate)
print('Number of channels:', nchannels)
print('Channel names:', channel_names)
print('Number of events:', len(event_onsets))
print('Event codes:', np.unique(event_codes))
print('Class labels:', cl_lab)
print('Number of classes:', nclasses)

Shape of EEG: (60, 8488200)
Sample rate: 2500
Number of channels: 60
Channel names: ['Fp1', 'AF7', 'AF3', 'AFz', 'F7', 'F5', 'F3', 'F1', 'Fz', 'FT7', 'FC5', 'FC3', 'FC1', 'T7', 'C5', 'C3', 'C1', 'Cz', 'TP7', 'CP5', 'CP3', 'CP1', 'CPz', 'P7', 'P5', 'P3', 'P1', 'Pz', 'PO7', 'PO3', 'POz', 'Fp2', 'AF4', 'AF8', 'F2', 'F4', 'F6', 'F8', 'FC2', 'FC4', 'FC6', 'FT8', 'C2', 'C4', 'C6', 'T8', 'CP2', 'CP4', 'CP6', 'TP8', 'P2', 'P4', 'P6', 'P8', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']
Number of events: 1
Event codes: [ 8 11 21 31 41 51 61]
Class labels: ['Forward', 'Backward', 'Left', 'Right', 'Up', 'Down', 'Rest']
Number of classes: 7


In [82]:
cd=[11, 21, 31, 41, 51, 61,8]
cl_lab=[ 'Forward','Backward','Left', 'Right','Up', 'Down','Rest']

In [83]:
# Dictionary to store the trials in, each class gets an entry
trials = {}

# The time window (in samples) to extract for each trial, here 0.5 -- 2.5 seconds
win = np.arange(int(0.5*sample_rate), int(2.5*sample_rate))

# Length of the time window
nsamples = len(win)

# Loop over the classes (right, foot)
for cl, code in zip(cl_lab, cd):
    
    # Extract the onsets for the class
    cl_onsets = event_onsets[event_codes == code]
    
    # Allocate memory for the trials
    trials[cl] = np.zeros((nchannels, nsamples, len(cl_onsets)))
    
    # Extract each trial
    for i, onset in enumerate(cl_onsets):
        trials[cl][:,:,i] = EEG[:, win+onset]

In [84]:
cl1=cl_lab[0]
cl2=cl_lab[1]
cl3=cl_lab[2]
cl4=cl_lab[3]
cl5=cl_lab[4]
cl6=cl_lab[5]

In [85]:
labels[0, event_onsets] = event_codes

In [86]:
#Apply the function
trials_filt = {cl1: bandpass(trials[cl1], 8, 30, sample_rate),
               cl2: bandpass(trials[cl2], 8, 30, sample_rate),
               cl3: bandpass(trials[cl3], 8, 30, sample_rate),
               cl4: bandpass(trials[cl4], 8, 30, sample_rate),
               cl5: bandpass(trials[cl5], 8, 30, sample_rate),
               cl6: bandpass(trials[cl6], 8, 30, sample_rate)}

In [87]:
trials_filt[cl1].shape

(60, 5000, 50)

In [88]:
#Percentage of trials to use for training (50-50 split here)
train_percentage = 0.5

# Calculate the number of trials for each class the above percentage boils down to
ntrain_1 = int(trials_filt[cl1].shape[2] * train_percentage)
ntrain_2 = int(trials_filt[cl2].shape[2] * train_percentage)
ntrain_3=  int(trials_filt[cl3].shape[2] * train_percentage)
ntrain_4 = int(trials_filt[cl4].shape[2] * train_percentage)
ntrain_5 = int(trials_filt[cl5].shape[2] * train_percentage)
ntrain_6 = int(trials_filt[cl6].shape[2] * train_percentage)




ntest_1 = trials_filt[cl1].shape[2] - ntrain_1
ntest_2 = trials_filt[cl2].shape[2] - ntrain_2
ntest_3 = trials_filt[cl3].shape[2] - ntrain_3
ntest_4 = trials_filt[cl4].shape[2] - ntrain_4
ntest_5 = trials_filt[cl5].shape[2] - ntrain_5
ntest_6 = trials_filt[cl6].shape[2] - ntrain_6


# Splitting the frequency filtered signal into a train and test set
train = {cl1: trials_filt[cl1][:,:,:ntrain_1],
         cl2: trials_filt[cl2][:,:,:ntrain_2],
         cl3: trials_filt[cl3][:,:,:ntrain_3],
         cl4: trials_filt[cl4][:,:,:ntrain_4],
         cl5: trials_filt[cl5][:,:,:ntrain_5],
         cl6: trials_filt[cl6][:,:,:ntrain_6],}

test = {cl1: trials_filt[cl1][:,:,ntrain_1:],
        cl2: trials_filt[cl2][:,:,ntrain_2:],
        cl3: trials_filt[cl3][:,:,ntrain_3:],
        cl4: trials_filt[cl4][:,:,ntrain_4:],
        cl5: trials_filt[cl5][:,:,ntrain_5:],
        cl6: trials_filt[cl6][:,:,ntrain_6:],}
    
#print (train.items)
# Train the CSP on the training set only
W = csp(train[cl1], train[cl2])

# Apply the CSP on both the training and test set
train[cl1] = apply_mix(W, train[cl1])
train[cl2] = apply_mix(W, train[cl2])
test[cl1] = apply_mix(W, test[cl1])
test[cl2] = apply_mix(W, test[cl2])

# Select only the first and last components for classification
# comp = np.array([0,-1])
# train[cl1] = train[cl1][comp,:,:]
# train[cl2] = train[cl2][comp,:,:]
# test[cl1] = test[cl1][comp,:,:]
# test[cl2] = test[cl2][comp,:,:]
# print (train[cl2].shape)

# Calculate the log-var
# train[cl1] = logvar(train[cl1])
# train[cl2] = logvar(train[cl2])
# test[cl1] = logvar(test[cl1])
# test[cl2] = logvar(test[cl2])
# print (train[cl2].shape)

In [89]:
ytrain=[]
xtrain=[]
ytest=[]
xtest=[]
events=[ 'Forward','Backward']

for l in range(len(events)):
    for j in range(train[events[l]].shape[2]):
      ytrain.append(l)
    xtrain.append(np.moveaxis(train[events[l]],-1,0))


for l in range(len(events)):
    for j in range(test[events[l]].shape[2]):
      ytest.append(l)
    xtest.append(np.moveaxis(test[events[l]],-1,0))

xtrain=np.vstack(xtrain)
print (xtrain.shape)
xtest=np.vstack(xtest)
xtest.shape

(50, 60, 5000)


(50, 60, 5000)

In [90]:
Y_train = np.array(ytrain)
Y_test = np.array(ytest)
Y_test.shape
X_train = np.transpose(xtrain, (0, 2, 1))
X_test = np.transpose(xtest,(0, 2, 1))
X_test.shape , Y_train.shape

((50, 5000, 60), (50,))

In [91]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D,BatchNormalization,LeakyReLU,MaxPool1D,\
GlobalAveragePooling1D,Dense,Dropout,AveragePooling1D
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model ,Sequential
from tensorflow.keras.constraints import MaxNorm
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras import regularizers
from keras import optimizers

In [92]:
def CNNModel():
    
    clear_session()
    model_m =Sequential()

    model_m.add(Dropout(0.50, input_shape=(X_train.shape[1],X_train.shape[-1])))
    model_m.add(Conv1D(128, 4, kernel_constraint = MaxNorm(1.),padding = 'same'))
    model_m.add(Conv1D(128, 4, kernel_constraint = MaxNorm(1.),padding = 'same',))
    model_m.add(BatchNormalization(axis = -1))
    model_m.add(Activation("relu"))
    model_m.add(MaxPool1D(2))
    model_m.add(Dropout(0.50))
    model_m.add(Conv1D(128, 10, kernel_constraint = MaxNorm(1.),padding = 'same'))
    model_m.add(Conv1D(128, 10, kernel_constraint = MaxNorm(1.),padding = 'same'))
    model_m.add(BatchNormalization(axis=-1))
    model_m.add(Activation("relu"))
    model_m.add(MaxPool1D(10))
    model_m.add(Flatten())
    model_m.add(Dropout(0.50))
    model_m.add(Dense(64, kernel_regularizer = regularizers.l1_l2(l1=0, l2=1e-5), activation = 'relu', kernel_constraint = MaxNorm(1.)))
    model_m.add(Dense(2, kernel_regularizer = regularizers.l1_l2(l1=0, l2=1e-5),activation = 'softmax',kernel_constraint = MaxNorm(1.)))

    return model_m


In [93]:
model=CNNModel()
optim=tf.keras.optimizers.SGD(learning_rate=0.001) #SGD #reomve rest #shuffle  #classes one by one
#model.compile(loss='sparse_categorical_crossentropy', optimizer = optimizers.Adam(lr = 0.001), metrics = ["acc"])
model.compile(optimizer =optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Epochs = 15

history = model.fit(X_train,Y_train,
                    epochs=Epochs,
                    validation_data=(X_test,Y_test),
                    batch_size=16,
                    verbose=1)

Epoch 1/15
4/4 [==============================] - 19s 4s/step - loss: 2.1517 - accuracy: 0.5600 - val_loss: 0.8059 - val_accuracy: 0.5000
Epoch 2/15
4/4 [==============================] - 16s 4s/step - loss: 1.4997 - accuracy: 0.4600 - val_loss: 0.7003 - val_accuracy: 0.4600
Epoch 3/15
4/4 [==============================] - 16s 4s/step - loss: 0.7104 - accuracy: 0.5000 - val_loss: 0.7019 - val_accuracy: 0.2600
Epoch 4/15
4/4 [==============================] - 15s 4s/step - loss: 0.7584 - accuracy: 0.4200 - val_loss: 0.7058 - val_accuracy: 0.5000
Epoch 5/15
4/4 [==============================] - 15s 4s/step - loss: 0.8815 - accuracy: 0.4400 - val_loss: 0.7019 - val_accuracy: 0.4000
Epoch 6/15
4/4 [==============================] - 15s 4s/step - loss: 0.6677 - accuracy: 0.6200 - val_loss: 0.6997 - val_accuracy: 0.5000
Epoch 7/15
4/4 [==============================] - 16s 4s/step - loss: 0.7245 - accuracy: 0.5600 - val_loss: 0.6998 - val_accuracy: 0.5000
Epoch 8/15
4/4 [==================

In [94]:
X_train=np.expand_dims(X_train,axis=-1)
X_train=np.moveaxis(X_train,1,2)
X_test=np.expand_dims(X_test,axis=-1)
X_test=np.moveaxis(X_test,1,2)
X_train.shape 

(50, 60, 5000, 1)

In [95]:
"""
 ARL_EEGModels - A collection of Convolutional Neural Network models for EEG
 Signal Processing and Classification, using Keras and Tensorflow
 Requirements:
    (1) tensorflow == 2.X (as of this writing, 2.0 - 2.3 have been verified
        as working)
 
 To run the EEG/MEG ERP classification sample script, you will also need
    (4) mne >= 0.17.1
    (5) PyRiemann >= 0.2.5
    (6) scikit-learn >= 0.20.1
    (7) matplotlib >= 2.2.3
    
 To use:
    
    (1) Place this file in the PYTHONPATH variable in your IDE (i.e.: Spyder)
    (2) Import the model as
        
        from EEGModels import EEGNet    
        
        model = EEGNet(nb_classes = ..., Chans = ..., Samples = ...)
        
    (3) Then compile and fit the model
    
        model.compile(loss = ..., optimizer = ..., metrics = ...)
        fitted    = model.fit(...)
        predicted = model.predict(...)
 Portions of this project are works of the United States Government and are not
 subject to domestic copyright protection under 17 USC Sec. 105.  Those 
 portions are released world-wide under the terms of the Creative Commons Zero 
 1.0 (CC0) license.  
 
 Other portions of this project are subject to domestic copyright protection 
 under 17 USC Sec. 105.  Those portions are licensed under the Apache 2.0 
 license.  The complete text of the license governing this material is in 
 the file labeled LICENSE.TXT that is a part of this project's official 
 distribution. 
"""

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import tensorflow as tf

def EEGNet(nb_classes, Chans = 64, Samples = 128, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
              D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    """ Keras Implementation of EEGNet
    http://iopscience.iop.org/article/10.1088/1741-2552/aace8c/meta
    Note that this implements the newest version of EEGNet and NOT the earlier
    version (version v1 and v2 on arxiv). We strongly recommend using this
    architecture as it performs much better and has nicer properties than
    our earlier version. For example:
        
        1. Depthwise Convolutions to learn spatial filters within a 
        temporal convolution. The use of the depth_multiplier option maps 
        exactly to the number of spatial filters learned within a temporal
        filter. This matches the setup of algorithms like FBCSP which learn 
        spatial filters within each filter in a filter-bank. This also limits 
        the number of free parameters to fit when compared to a fully-connected
        convolution. 
        
        2. Separable Convolutions to learn how to optimally combine spatial
        filters across temporal bands. Separable Convolutions are Depthwise
        Convolutions followed by (1x1) Pointwise Convolutions. 
        
    
    While the original paper used Dropout, we found that SpatialDropout2D 
    sometimes produced slightly better results for classification of ERP 
    signals. However, SpatialDropout2D significantly reduced performance 
    on the Oscillatory dataset (SMR, BCI-IV Dataset 2A). We recommend using
    the default Dropout in most cases.
        
    Assumes the input signal is sampled at 128Hz. If you want to use this model
    for any other sampling rate you will need to modify the lengths of temporal
    kernels and average pooling size in blocks 1 and 2 as needed (double the 
    kernel lengths for double the sampling rate, etc). Note that we haven't 
    tested the model performance with this rule so this may not work well. 
    
    The model with default parameters gives the EEGNet-8,2 model as discussed
    in the paper. This model should do pretty well in general, although it is
  	advised to do some model searching to get optimal performance on your
  	particular dataset.
    We set F2 = F1 * D (number of input filters = number of output filters) for
    the SeparableConv2D layer. We haven't extensively tested other values of this
    parameter (say, F2 < F1 * D for compressed learning, and F2 > F1 * D for
    overcomplete). We believe the main parameters to focus on are F1 and D. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. We found
                        that setting this to be half the sampling rate worked
                        well in practice. For the SMR dataset in particular
                        since the data was high-passed at 4Hz we used a kernel
                        length of 32.     
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D. 
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.

    """
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)




def EEGNet_SSVEP(nb_classes =12, Chans =8 , Samples =256,
             dropoutRate = 0.5, kernLength = 256, F1 = 96, 
             D = 1, F2 = 96, dropoutType = 'Dropout'):
    """ SSVEP Variant of EEGNet, as used in [1]. 
    Inputs:
        
      nb_classes      : int, number of classes to classify
      Chans, Samples  : number of channels and time points in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. 
      D               : number of spatial filters to learn within each temporal
                        convolution.
      dropoutType     : Either SpatialDropout2D or Dropout, passed as a string.
      
      
    [1]. Waytowich, N. et. al. (2018). Compact Convolutional Neural Networks
    for Classification of Asynchronous Steady-State Visual Evoked Potentials.
    Journal of Neural Engineering vol. 15(6). 
    http://iopscience.iop.org/article/10.1088/1741-2552/aae5d8
    """
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (Chans, Samples, 1))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (Chans, Samples, 1),
                                   use_bias = False)(input1)
    block1       = BatchNormalization()(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.))(block1)
    block1       = BatchNormalization()(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4))(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization()(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8))(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)



def EEGNet_old(nb_classes, Chans = 64, Samples = 128, regRate = 0.0001,
           dropoutRate = 0.25, kernels = [(2, 32), (8, 4)], strides = (2, 4)):
    """ Keras Implementation of EEGNet_v1 (https://arxiv.org/abs/1611.08024v2)
    This model is the original EEGNet model proposed on arxiv
            https://arxiv.org/abs/1611.08024v2
    
    with a few modifications: we use striding instead of max-pooling as this 
    helped slightly in classification performance while also providing a 
    computational speed-up. 
    
    Note that we no longer recommend the use of this architecture, as the new
    version of EEGNet performs much better overall and has nicer properties.
    
    Inputs:
        
        nb_classes     : total number of final categories
        Chans, Samples : number of EEG channels and samples, respectively
        regRate        : regularization rate for L1 and L2 regularizations
        dropoutRate    : dropout fraction
        kernels        : the 2nd and 3rd layer kernel dimensions (default is 
                         the [2, 32] x [8, 4] configuration)
        strides        : the stride size (note that this replaces the max-pool
                         used in the original paper)
    
    """

    # start the model
    input_main   = Input((Chans, Samples))
    layer1       = Conv2D(16, (Chans, 1), input_shape=(Chans, Samples, 1),
                                 kernel_regularizer = l1_l2(l1=regRate, l2=regRate))(input_main)
    layer1       = BatchNormalization()(layer1)
    layer1       = Activation('elu')(layer1)
    layer1       = Dropout(dropoutRate)(layer1)
    
    permute_dims = 2, 1, 3
    permute1     = Permute(permute_dims)(layer1)
    
    layer2       = Conv2D(4, kernels[0], padding = 'same', 
                            kernel_regularizer=l1_l2(l1=0.0, l2=regRate),
                            strides = strides)(permute1)
    layer2       = BatchNormalization()(layer2)
    layer2       = Activation('elu')(layer2)
    layer2       = Dropout(dropoutRate)(layer2)
    
    layer3       = Conv2D(4, kernels[1], padding = 'same',
                            kernel_regularizer=l1_l2(l1=0.0, l2=regRate),
                            strides = strides)(layer2)
    layer3       = BatchNormalization()(layer3)
    layer3       = Activation('elu')(layer3)
    layer3       = Dropout(dropoutRate)(layer3)
    
    flatten      = Flatten(name = 'flatten')(layer3)
    
    dense        = Dense(nb_classes, name = 'dense')(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)



def DeepConvNet(nb_classes, Chans = 64, Samples = 256,
                dropoutRate = 0.5):
    """ Keras implementation of the Deep Convolutional Network as described in
    Schirrmeister et. al. (2017), Human Brain Mapping.
    
    This implementation assumes the input is a 2-second EEG signal sampled at 
    128Hz, as opposed to signals sampled at 250Hz as described in the original
    paper. We also perform temporal convolutions of length (1, 5) as opposed
    to (1, 10) due to this sampling rate difference. 
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                      ours        original paper
    pool_size        1, 2        1, 3
    strides          1, 2        1, 3
    conv filters     1, 5        1, 10
    
    Note that this implementation has not been verified by the original 
    authors. 
    
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(25, (1, 5), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1       = Activation('elu')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)
  
    block2       = Conv2D(50, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block2       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block2)
    block2       = Activation('elu')(block2)
    block2       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)
    
    block3       = Conv2D(100, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block2)
    block3       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block3)
    block3       = Activation('elu')(block3)
    block3       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block3)
    block3       = Dropout(dropoutRate)(block3)
    
    block4       = Conv2D(200, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(block3)
    block4       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block4)
    block4       = Activation('elu')(block4)
    block4       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block4)
    block4       = Dropout(dropoutRate)(block4)
    
    flatten      = Flatten()(block4)
    
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)


# need these for ShallowConvNet
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   


def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    """ Keras implementation of the Shallow Convolutional Network as described
    in Schirrmeister et. al. (2017), Human Brain Mapping.
    
    Assumes the input is a 2-second EEG signal sampled at 128Hz. Note that in 
    the original paper, they do temporal convolutions of length 25 for EEG
    data sampled at 250Hz. We instead use length 13 since the sampling rate is 
    roughly half of the 250Hz which the paper used. The pool_size and stride
    in later layers is also approximately half of what is used in the paper.
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                     ours        original paper
    pool_size        1, 35       1, 75
    strides          1, 7        1, 15
    conv filters     1, 13       1, 25    
    
    Note that this implementation has not been verified by the original 
    authors. We do note that this implementation reproduces the results in the
    original paper with minor deviations. 
    """

    # start the model
    input_main   = Input((Chans, Samples, 1))
    block1       = Conv2D(40, (1, 13), 
                                 input_shape=(Chans, Samples, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)))(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)))(block1)
    block1       = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7))(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)


In [96]:
num_classes =2
kernLength=10
model =EEGNet(nb_classes=num_classes, Chans = X_train.shape[1], Samples = X_train.shape[2],kernLength=kernLength,dropoutRate=0.5)
#model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',  metrics=['accuracy'])
optim=tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer=optim,loss='sparse_categorical_crossentropy',metrics=['accuracy'])
Epochs=8
history =  model.fit(X_train,Y_train,
                    epochs=Epochs,
                    validation_data=(X_test,Y_test),
                    batch_size=8,
                    verbose=1)

Epoch 1/8
7/7 [==============================] - 15s 2s/step - loss: 0.7102 - accuracy: 0.5600 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 2/8
7/7 [==============================] - 13s 2s/step - loss: 0.6445 - accuracy: 0.7200 - val_loss: 0.6923 - val_accuracy: 0.5200
Epoch 3/8
7/7 [==============================] - 17s 2s/step - loss: 0.6371 - accuracy: 0.6000 - val_loss: 0.6922 - val_accuracy: 0.5200
Epoch 4/8
7/7 [==============================] - 15s 2s/step - loss: 0.6178 - accuracy: 0.7600 - val_loss: 0.6923 - val_accuracy: 0.5200
Epoch 5/8
7/7 [==============================] - 14s 2s/step - loss: 0.5797 - accuracy: 0.8000 - val_loss: 0.6923 - val_accuracy: 0.5600
Epoch 6/8
7/7 [==============================] - 15s 2s/step - loss: 0.5302 - accuracy: 0.9600 - val_loss: 0.6923 - val_accuracy: 0.5200
Epoch 7/8
7/7 [==============================] - 15s 2s/step - loss: 0.5151 - accuracy: 0.9600 - val_loss: 0.6922 - val_accuracy: 0.5200
Epoch 8/8
7/7 [==========================